In [1]:
# create the configuration of the low-fidelity model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

# get the accuracy metrics
from sklearn.metrics import r2_score

from mfbml.metrics import (
    mean_log_likelihood_value,
    normalized_mae,
    normalized_rmse,
)
from mfbml.methods.bayes_neural_nets import BNNWrapper

# fix the random seed for reproducibility
seed = 1999
np.random.seed(seed)
torch.manual_seed(seed)

In [3]:
# read the data
data = pd.read_pickle("../data_generation/data.pkl")
# extract dataset
samples = data["samples"]
hf_samples = samples[0]
lf_samples = samples[1]
hf_responses = data["responses_lf1"][0]
responses_lf1 = data["responses_lf1"]
responses_lf2 = data["responses_lf2"]
responses_lf3 = data["responses_lf3"]
test_samples = data["test_samples"]
test_hf_responses_noiseless = data["test_hf_responses_noiseless"]
test_hf_responses = data["test_hf_responses"]
test_lf1_responses = data["test_lf1_responses"]
test_lf2_responses = data["test_lf2_responses"]
test_lf3_responses = data["test_lf3_responses"]

In [4]:
# normalize the responses
hf_responses_mean = hf_responses.mean().numpy()
hf_responses_std = hf_responses.std().numpy()
hf_responses_scaled = (hf_responses.clone() -
                       hf_responses_mean) / hf_responses_std
sigma_scale = float(0.05 / hf_responses_std)
# create the sf_bnn model
for ii in [50, 128, 256, 512, 1024]:

    bnn_model = BNNWrapper(
        in_features=1,
        hidden_features=[ii, ii],
        out_features=1,
        activation="Tanh",
        lr=0.001,
        sigma=sigma_scale,
    )
    # train the model
    bnn_model.train(
        X=hf_samples,
        Y=hf_responses_scaled,
        num_epochs=30000,
        sample_freq=100,
        burn_in_epochs=20000,
    )
    # save the model
    with open(f"sf_bnn_model_{ii}_{ii}.pkl", "wb") as f:
        torch.save(bnn_model, f)

    (bnn_y,
     bnn_epistemic,
     bnn_total_unc,
     bnn_aleatoric) = bnn_model.predict(
        X=test_samples)
    bnn_y = bnn_y * hf_responses_std + hf_responses_mean
    bnn_total_unc = bnn_total_unc * hf_responses_std
    bnn_epistemic = bnn_epistemic * hf_responses_std
    bnn_aleatoric = bnn_aleatoric * hf_responses_std
    plt.figure()
    plt.plot(hf_samples, hf_responses, "o", label="hf")
    plt.plot(test_samples.numpy(), bnn_y, label="hf prediction")

    plt.plot(
        test_samples.numpy(),
        test_hf_responses_noiseless.numpy(),
        label="hf ground truth",
    )
    plt.fill_between(
        test_samples.flatten().numpy(),
        (bnn_y - 2 * bnn_total_unc).reshape(-1),
        (bnn_y + 2 * bnn_total_unc).reshape(-1),
        alpha=0.5,
        label="uncertainty",
    )
    # set the ylim
    plt.ylim([-1.75, 1.75])
    plt.legend()
    # plt.show()
    plt.savefig(f"sf_bnn_{ii}_{ii}.png")
    plt.close()

/home/yaga/Documents/GitHub/mfbml/src/mfbml/inference/psgld.py:108: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1485.)
  V.mul_(alpha).addcmul_(1 - alpha, d_p, d_p)


epoch:     0/30000
nll_loss: 295.683, prior_loss: 2506.492, total: 2802.174
noise: 0.126
epoch:   100/30000
nll_loss: 230.086, prior_loss: 2729.678, total: 2959.764
noise: 0.126
epoch:   200/30000
nll_loss: 229.922, prior_loss: 2845.079, total: 3075.001
noise: 0.126
epoch:   300/30000
nll_loss: 227.036, prior_loss: 2944.154, total: 3171.189
noise: 0.126
epoch:   400/30000
nll_loss: 221.975, prior_loss: 3037.107, total: 3259.082
noise: 0.126
epoch:   500/30000
nll_loss: 219.274, prior_loss: 3109.088, total: 3328.363
noise: 0.126
epoch:   600/30000
nll_loss: 208.896, prior_loss: 3187.877, total: 3396.773
noise: 0.126
epoch:   700/30000
nll_loss: 195.192, prior_loss: 3274.125, total: 3469.316
noise: 0.126
epoch:   800/30000
nll_loss: 200.328, prior_loss: 3338.168, total: 3538.496
noise: 0.126
epoch:   900/30000
nll_loss: 193.369, prior_loss: 3396.001, total: 3589.370
noise: 0.126
epoch:  1000/30000
nll_loss: 183.350, prior_loss: 3467.325, total: 3650.676
noise: 0.126
epoch:  1100/30000
nl